In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

def simulate_sentiment(tickers, days):
    return pd.DataFrame(np.random.randn(days, len(tickers)), columns=tickers)

def calculate_volatility(returns, window=20):
    return returns.rolling(window=window).std() * np.sqrt(252)

def sentiment_volatility_strategy(sentiment_data, price_data, volatility_data, n_components=3, volatility_threshold=0.5, min_hold_period=5):
    scaler = StandardScaler()
    scaled_sentiment = scaler.fit_transform(sentiment_data)
    
    pca = PCA(n_components=n_components)
    sentiment_factors = pca.fit_transform(scaled_sentiment)
    
    # Reconstruct the sentiment scores using PCA components
    reconstructed_sentiment = pca.inverse_transform(sentiment_factors)
    sentiment_score = pd.DataFrame(reconstructed_sentiment, index=price_data.index, columns=price_data.columns)
    
    # Long positions: positive sentiment and low volatility
    long_signals = (sentiment_score > 0) & (volatility_data < volatility_data.quantile(volatility_threshold))
    
    # Short positions: negative sentiment and high volatility
    short_signals = (sentiment_score < 0) & (volatility_data > volatility_data.quantile(1 - volatility_threshold))
    
    long_positions = long_signals.rolling(window=min_hold_period).min().fillna(False).astype(bool)
    short_positions = short_signals.rolling(window=min_hold_period).min().fillna(False).astype(bool)
    
    position_size = 1 / len(price_data.columns)
    long_returns = price_data.pct_change().where(long_positions, 0) * position_size
    short_returns = -price_data.pct_change().where(short_positions, 0) * position_size
    strategy_returns = long_returns.sum(axis=1) + short_returns.sum(axis=1)
    
    return strategy_returns, long_positions, short_positions, sentiment_score

# Fetch stock data
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META']
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)
price_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

sentiment_data = simulate_sentiment(tickers, len(price_data))
returns = price_data.pct_change()
volatility_data = calculate_volatility(returns)

strategy_returns, long_positions, short_positions, sentiment_score = sentiment_volatility_strategy(sentiment_data, price_data, volatility_data)

cumulative_returns = (1 + strategy_returns).cumprod()

# Create interactive plot
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Strategy Cumulative Returns', 'Sentiment Scores', 'Long Positions', 'Short Positions'))

fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, name='Strategy Returns'), row=1, col=1)

for ticker in tickers:
    fig.add_trace(go.Scatter(x=sentiment_score.index, y=sentiment_score[ticker], name=f'{ticker} Sentiment'), row=2, col=1)
    fig.add_trace(go.Scatter(x=long_positions.index, y=long_positions[ticker] * 1, name=f'{ticker} Long', mode='markers'), row=3, col=1)
    fig.add_trace(go.Scatter(x=short_positions.index, y=short_positions[ticker] * 1, name=f'{ticker} Short', mode='markers'), row=4, col=1)

fig.update_layout(height=1200, title_text="Sentiment-Driven Volatility Strategy")
fig.show()

# Print strategy performance
total_return = cumulative_returns.iloc[-1] - 1
annualized_return = (cumulative_returns.iloc[-1] ** (252 / len(cumulative_returns)) - 1) if len(cumulative_returns) > 0 else 0
sharpe_ratio = (strategy_returns.mean() / strategy_returns.std()) * np.sqrt(252) if strategy_returns.std() != 0 else 0

print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


[*********************100%***********************]  5 of 5 completed


Total Return: -2.93%
Annualized Return: -1.48%
Sharpe Ratio: -0.93
